In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from pathlib import Path

from generation import generate_agents_subpopulations, generate_agents_berlin_trips, generate_trips_berlinwise, \
    berlinwise_col, berlin_transit_col, generate_trips_berlinwise, generate_nonberlin_trips
from reader import read_agents, read_trips, read_trips_berlinwise
from scenarios import BerlinScenario, SCENARIOS_PATH

In [3]:
def path(scenario: BerlinScenario, kind, file_name_suffix) -> Path:
    return SCENARIOS_PATH / scenario.value / kind / f"{scenario.value}.{file_name_suffix}"

Read in input csv (both from MATSim output and from Java analysis)

In [4]:
agents = read_agents(path(BerlinScenario.BASE, "output", "output_persons.csv.gz"))
agents

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone
person,,,,,,,
10000001,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg
1000001,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg
100002101,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg
100002901,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg
100003501,130.202918,4.625913e+06,5.824592e+06,home_82200.0,1847.0,person,brandenburg
...,...,...,...,...,...,...,...
freight_997_87464-berlin,117.274442,4.603503e+06,5.808240e+06,freight,NaN,freight,NaN
freight_998_87464-berlin,116.469365,4.603503e+06,5.808240e+06,freight,NaN,freight,NaN
freight_999_87464-berlin,118.242244,4.603503e+06,5.808240e+06,freight,NaN,freight,NaN


In [5]:
trips_by_scenario = {scenario: read_trips(path(BerlinScenario.BASE, "output", "output_trips.csv.gz")) for
                     scenario in BerlinScenario}
trips_by_scenario[BerlinScenario.BASE]

,person,trip_number,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,modes,...,start_facility_id,start_link,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop
trip_id,,,,,,,,,,,,,,,,,,,,,
10000001_1,10000001,1,09:17:00,00:17:42,00:00:00,8050,6787,car,car,walk-car-walk,...,NaN,144395,4.602586e+06,5.705900e+06,NaN,134075,4.597352e+06,5.710221e+06,NaN,NaN
10000001_2,10000001,2,09:39:42,00:27:43,00:00:00,18437,13779,car,car,walk-car-walk,...,NaN,134075,4.597352e+06,5.710221e+06,NaN,77361,4.583822e+06,5.712834e+06,NaN,NaN
10000001_3,10000001,3,10:19:25,00:48:35,00:00:00,34241,24989,car,car,walk-car-walk,...,NaN,77361,4.583822e+06,5.712834e+06,NaN,31841,4.608361e+06,5.708108e+06,NaN,NaN
10000001_4,10000001,4,12:02:00,00:18:43,00:00:00,10508,5899,car,car,walk-car-walk,...,NaN,31841,4.608361e+06,5.708108e+06,NaN,144395,4.603071e+06,5.705496e+06,NaN,NaN
10000001_5,10000001,5,13:05:43,00:42:46,00:00:00,17661,9403,car,car,walk-car-walk,...,NaN,144395,4.603071e+06,5.705496e+06,NaN,106140,4.609164e+06,5.698333e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
freight_997_87464-berlin_1,freight_997_87464-berlin,1,06:01:16,00:26:07,00:00:00,13893,10437,freight,freight,walk-freight-walk,...,NaN,87167,4.603503e+06,5.808240e+06,NaN,54169,4.598759e+06,5.817537e+06,NaN,NaN
freight_998_87464-berlin_1,freight_998_87464-berlin,1,10:58:14,00:29:36,00:00:00,16478,10720,freight,freight,walk-freight-walk,...,NaN,87167,4.603503e+06,5.808240e+06,NaN,90464,4.593564e+06,5.812259e+06,NaN,NaN
freight_999_87464-berlin_1,freight_999_87464-berlin,1,02:54:09,00:21:21,00:00:00,10876,7147,freight,freight,walk-freight-walk,...,NaN,87167,4.603503e+06,5.808240e+06,NaN,88992,4.604471e+06,5.815321e+06,NaN,NaN


Generate temporary files that split the population into relevant, freight and ride

In [21]:
agent_subpopulations = generate_agents_subpopulations(agents, trips_by_scenario[BerlinScenario.BASE])
agent_subpopulations.relevant.to_csv(path(BerlinScenario.BASE, "analysis", ".relevant_persons.csv"), sep=";")
agent_subpopulations.freight.to_csv(path(BerlinScenario.BASE, "analysis", ".freight_persons.csv"), sep=";")
agent_subpopulations.ride.to_csv(path(BerlinScenario.BASE, "analysis", ".ride_persons.csv"), sep=";")
agent_subpopulations.relevant

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes
person,,,,,,,,
10000001,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg,{car}
1000001,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg,{bicycle}
100002101,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg,{bicycle}
100002901,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg,{car}
100004901,110.151148,4.625238e+06,5.824173e+06,home_54600.0,2070.0,person,brandenburg,{car}
...,...,...,...,...,...,...,...,...
99994101,95.497982,4.632244e+06,5.826463e+06,home_60600.0,4329.0,person,brandenburg,{bicycle}
99995401,117.394194,4.631520e+06,5.826864e+06,home_83400.0,1142.0,person,brandenburg,{bicycle}
99995901,132.648340,4.627167e+06,5.827323e+06,home_77400.0,2332.0,person,brandenburg,"{bicycle, walk}"


Generate `trips_berlinwise.csv`

In [7]:
trips_berlinwise_by_scenario = {scenario: read_trips_berlinwise(path(scenario, "analysis", "trips_berlinwise.csv")) for
                                scenario in BerlinScenario}

Generate `ALL.trips_berlinwise.csv`

In [14]:
trips_berlinwise = generate_trips_berlinwise(agent_subpopulations.relevant, trips_berlinwise_by_scenario)
trips_berlinwise.to_csv(SCENARIOS_PATH / "ALL.trips_berlinwise.csv", sep=";", index=False)
trips_berlinwise

,trip_id,person,trip_number,0.BASE.berlinwise,1.GR-HS.berlinwise,2.GR-WS.berlinwise,3.KR-HS.berlinwise,4.KB.berlinwise,5.MV.berlinwise,6.S1.berlinwise,7.S2.berlinwise,8.S3.berlinwise
10000001_1,10000001_1,10000001,1.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_2,10000001_2,10000001,2.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_3,10000001_3,10000001,3.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_4,10000001_4,10000001,4.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_5,10000001_5,10000001,5.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
...,...,...,...,...,...,...,...,...,...,...,...,...
99996201_3,99996201_3,99996201,3.0,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner
99996201_4,99996201_4,99996201,4.0,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig
99998601_1,99998601_1,99998601,1.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
99998601_2,99998601_2,99998601,2.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin


Listet trips, die unterschiedliche berlinwise-Werte haben (z.B. von berlin_orig auf berlin_inner wechseln)

In [31]:
def berlinwise_differs_across_scenarios(row):
    values = {row[berlinwise_col(scenario)] for scenario in BerlinScenario}
    values.discard("cancelled")
    return len(values) != 1


trips_berlinwise[trips_berlinwise.apply(berlinwise_differs_across_scenarios, axis=1)]

,trip_id,person,trip_number,0.BASE.berlinwise,1.GR-HS.berlinwise,2.GR-WS.berlinwise,3.KR-HS.berlinwise,4.KB.berlinwise,5.MV.berlinwise,6.S1.berlinwise,7.S2.berlinwise,8.S3.berlinwise
100508401_1,100508401_1,100508401,1.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,berlin_dest,non_berlin,non_berlin,non_berlin
100512701_6,100512701_6,100512701,6.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,berlin_dest,non_berlin,non_berlin,non_berlin
100620201_9,100620201_9,100620201,9.0,berlin_dest,berlin_dest,berlin_dest,berlin_dest,berlin_dest,non_berlin,berlin_dest,berlin_dest,berlin_dest
10165601_2,10165601_2,10165601,2.0,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_inner,berlin_orig,berlin_orig,berlin_orig
101908701_4,101908701_4,101908701,4.0,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,berlin_dest,non_berlin,non_berlin,non_berlin
...,...,...,...,...,...,...,...,...,...,...,...,...
98491001_3,98491001_3,98491001,3.0,berlin_orig,berlin_orig,berlin_orig,berlin_orig,cancelled,berlin_inner,berlin_orig,berlin_orig,berlin_orig
98865501_2,98865501_2,98865501,2.0,berlin_dest,berlin_dest,berlin_dest,berlin_dest,berlin_dest,non_berlin,berlin_dest,berlin_dest,berlin_dest
99395801_4,99395801_4,99395801,4.0,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_inner,berlin_orig,berlin_orig,berlin_orig
99504801_4,99504801_4,99504801,4.0,berlin_dest,berlin_dest,berlin_dest,berlin_dest,berlin_dest,non_berlin,berlin_dest,berlin_dest,berlin_dest


Generate `ALL.nonberlin_trips.csv`

In [17]:
nonberlin_trips = generate_nonberlin_trips(agent_subpopulations.relevant, trips_by_scenario, trips_berlinwise_by_scenario)
nonberlin_trips.to_csv(SCENARIOS_PATH / "ALL.nonberlin_trips.csv", sep=";")
nonberlin_trips

,person,trip_number,0.BASE.berlin_transit,1.GR-HS.berlin_transit,2.GR-WS.berlin_transit,3.KR-HS.berlin_transit,4.KB.berlin_transit,5.MV.berlin_transit,6.S1.berlin_transit,7.S2.berlin_transit,8.S3.berlin_transit
trip_id,,,,,,,,,,,
10000001_1,10000001,1,other,other,other,other,other,other,other,other,other
10000001_2,10000001,2,other,other,other,other,other,other,other,other,other
10000001_3,10000001,3,other,other,other,other,other,other,other,other,other
10000001_4,10000001,4,other,other,other,other,other,other,other,other,other
10000001_5,10000001,5,other,other,other,other,other,other,other,other,other
...,...,...,...,...,...,...,...,...,...,...,...
99995901_4,99995901,4,other,other,other,other,other,other,other,other,other
99996201_1,99996201,1,other,other,other,other,other,other,other,other,other
99998601_1,99998601,1,other,other,other,other,other,other,other,other,other


Listet trips, die berlin_transit sind und einen teleportierten Modus (walk oder bicycle) haben.

In [23]:
a = trips_by_scenario[BerlinScenario.GR_HS].join(trips_berlinwise_by_scenario[BerlinScenario.GR_HS],
                                                 "trip_id", "outer", lsuffix="output", rsuffix="analysis")
trips_with_bicycle_transit = a[(a["berlinwise"] == "berlin_transit") & (a["main_mode"] == "bicycle")]

In [30]:
trips_berlinwise_by_scenario[BerlinScenario.BASE].join(trips_with_bicycle_transit[["start_linkoutput", "start_linkanalysis"]], "trip_id", "inner")

,person,trip_number,start_link,end_link,berlinwise,start_linkoutput,start_linkanalysis
trip_id,,,,,,,
108173201_3,108173201,3,14578,50131,non_berlin,14578,14578
108173201_4,108173201,4,50131,14578,non_berlin,50131,50131
118893501_1,118893501,1,105754,56594,non_berlin,105754,105754
124860301_3,124860301,3,2582,136217,non_berlin,2582,2582
126558801_1,126558801,1,98932,125423,non_berlin,98932,98932
...,...,...,...,...,...,...,...
95617401_2,95617401,2,141606,142972,non_berlin,141606,141606
95737701_5,95737701,5,141646,20968,non_berlin,141646,141646
95737701_6,95737701,6,20968,141646,non_berlin,20968,20968


Generates `agents_berlin_trips.csv`

In [20]:
agents_berlin_trips = generate_agents_berlin_trips(agent_subpopulations.relevant,
                                                   trips_berlinwise_by_scenario[BerlinScenario.BASE])
agents_berlin_trips.to_csv(SCENARIOS_PATH / "agents_berlin_trips.csv", sep=";", index=False)
agents_berlin_trips

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,at_least_one_berlin_trip
person,,,,,,,,,
10000001,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg,{car},False
1000001,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg,{bicycle},False
100002101,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg,{bicycle},True
100002901,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg,{car},False
100004901,110.151148,4.625238e+06,5.824173e+06,home_54600.0,2070.0,person,brandenburg,{car},False
...,...,...,...,...,...,...,...,...,...
99994101,95.497982,4.632244e+06,5.826463e+06,home_60600.0,4329.0,person,brandenburg,{bicycle},False
99995401,117.394194,4.631520e+06,5.826864e+06,home_83400.0,1142.0,person,brandenburg,{bicycle},False
99995901,132.648340,4.627167e+06,5.827323e+06,home_77400.0,2332.0,person,brandenburg,"{bicycle, walk}",False


Listet agents, die in der `home-activity-zone` berlin sind, aber gar keine trips in berlin haben. Hauptsächlich immobile Personen, aber auch 9 Grenzfälle

In [ ]:
# sanity check
agents_berlin_trips[
    (agents_berlin_trips["home-activity-zone"] == "berlin") & (~agents_berlin_trips["at_least_one_berlin_trip"])]

,person,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,at_least_one_berlin_trip
171751401,171751401,0.0,4.596946e+06,5.816937e+06,home_86400.0,2332.0,person,berlin,{},False
171753701,171753701,0.0,4.577484e+06,5.822459e+06,home_86400.0,1847.0,person,berlin,{},False
171754501,171754501,0.0,4.603655e+06,5.820810e+06,home_86400.0,1847.0,person,berlin,{},False
171755201,171755201,0.0,4.599184e+06,5.827074e+06,home_86400.0,1399.0,person,berlin,{},False
171798301,171798301,0.0,4.589880e+06,5.814495e+06,home_86400.0,1847.0,person,berlin,{},False
...,...,...,...,...,...,...,...,...,...,...
451412001,451412001,0.0,4.582536e+06,5.825134e+06,home_86400.0,4329.0,person,berlin,{},False
451461601,451461601,0.0,4.597971e+06,5.821096e+06,home_86400.0,3156.0,person,berlin,{},False
451484001,451484001,0.0,4.598753e+06,5.810959e+06,home_86400.0,3156.0,person,berlin,{},False
451485701,451485701,0.0,4.597367e+06,5.816051e+06,home_86400.0,1630.0,person,berlin,{},False
